# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [2]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations

from tqdm import tqdm_notebook
import pandas as pd

%autosave 5
%matplotlib inline

Autosaving every 5 seconds


## Fetching Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [2]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [3]:
historic_data = CoinMarketCap.historic()
historic_data.tail()

,date,open,high,low,close,volume,market_cap
2736,2013-05-02,116.38,125.60,92.28,105.21,0,1168517495
2737,2013-05-01,139.00,139.89,107.72,116.99,0,1298954594
2738,2013-04-30,144.00,146.93,134.05,139.00,0,1542813125
2739,2013-04-29,134.44,147.49,134.00,144.54,0,1603768865
2740,2013-04-28,135.30,135.98,132.10,134.21,0,1488566728


The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [4]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` variable.
#
model_data = historic_data[historic_data['date']>='2017-01-01']

In [5]:
model_data.tail()

,date,open,high,low,close,volume,market_cap
1392,2017-01-05,1156.73,1191.10,910.42,1013.38,510199008,16300254795
1393,2017-01-04,1044.40,1159.42,1044.40,1154.73,344945984,18571869009
1394,2017-01-03,1021.60,1044.08,1021.60,1043.84,185168000,16786368910
1395,2017-01-02,998.62,1031.39,996.70,1021.75,222184992,16429024775
1396,2017-01-01,963.66,1003.08,958.70,998.33,147775008,16050407461


## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [5]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [6]:
M.build()

In [7]:
M.train(epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 9s 9s/step - loss: 0.0303
Epoch 2/100
1/1 [==============================] - 0s 119ms/step - loss: 0.0282
Epoch 3/100
1/1 [==============================] - 0s 91ms/step - loss: 0.0268
Epoch 4/100
1/1 [==============================] - 0s 98ms/step - loss: 0.0256
Epoch 5/100
1/1 [==============================] - 0s 86ms/step - loss: 0.0245
Epoch 6/100
1/1 [==============================] - 0s 74ms/step - loss: 0.0235
Epoch 7/100
1/1 [==============================] - 0s 76ms/step - loss: 0.0226
Epoch 8/100
1/1 [==============================] - 0s 86ms/step - loss: 0.0218
Epoch 9/100
1/1 [==============================] - 0s 88ms/step - loss: 0.0209
Epoch 10/100
1/1 [==============================] - 0s 102ms/step - loss: 0.0201
Epoch 11/100
1/1 [==============================] - 0s 76ms/step - loss: 0.0194
Epoch 12/100
1/1 [==============================] - 0s 82ms/step - loss: 0.0186
Epoch 13/100
1/1 [==============================

Epoch 99/100
1/1 [==============================] - 0s 97ms/step - loss: 7.1027e-06
Epoch 100/100
1/1 [==============================] - 0s 93ms/step - loss: 8.6075e-06


We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [12]:
M.predict(denormalized=True)

array([13265.805, 15451.118, 15348.878, 15502.748, 15475.497, 15344.149,
       17214.115], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [13]:
M.evaluate()

{'mse': 0.0, 'rmse': 2404.62, 'mape': 16.2}

Finally, we can now save the trained model on disk for later use.

In [21]:
M.save('bitcoin_model_prod_v00.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [16]:
M = Model(path='bitcoin_model_prod_v00.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [17]:
M.predict(denormalized=True)

array([13400.709, 15491.156, 15486.53 , 15517.533, 15446.425, 15380.349,
       17265.262], dtype=float32)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [18]:
M1 = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [19]:
M1.build()

In [20]:
M1.train()

In [21]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
results = []
for i in range(1, 10 + 1):
    M1.train(model_data[i * 7:7 * (40 + i) + 7])
    results.append(M.evaluate())

In [22]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 42: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 43: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 44: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 45: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 46: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 47: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 48: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 49: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}
Week 50: {'mse': 0.0, 'rmse': 2284.86, 'mape': 15.31}


In [23]:
M1.predict(denormalized=True)

array([11763.159, 11781.231, 11705.402, 12098.664, 11777.905, 11765.833,
       12380.267], dtype=float32)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [24]:
old_data = model_data[0*7:7*48 + 7]

In [25]:
new_data = model_data[0*7:7*49 + 7]

In [29]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [30]:
M2.build()
M2.train()

In [31]:
M2.predict(denormalized=True)

array([13291.273 , 12702.707 , 12611.3545, 12435.209 , 13052.275 ,
       12854.446 , 13480.808 ], dtype=float32)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [32]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#
M3 = Model(data=new_data,
          variable='close',
          predicted_period_size=7)

In [33]:
M3.build()
M3.train()

In [34]:
M3.predict(denormalized=True)

array([13295.038, 13599.138, 13725.435, 14205.102, 14140.812, 14049.945,
       14425.465], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.